Ciclo Básico: A/B Testing
## 1.0. O problema de negócio.

## 2.0. O planejamento da solução. 

## 3.0. A motivação do Teste A/B 

## 4.0. PA Bond - Básico

## 5.0. Design de Experimento I

## 6.0. Design de Experimento II

## 7.0. Preparando os dados

## 8.0. Testando as hipóteses

## 9.0. Os tipos de testes de hipótese 

## 10.0.  Resumo do Ciclo Básico 

## 11.0.  Sessão de Cross-validation

# Ciclo Intermediário: Multivariate Testing Aula 1: O problema de negócio.

## - Aula 2: O planejamento da solução.

## - Aula 3: A motivação do Teste A/B/C/n

### 1. A empresa Electronic House
A Electronic House é um comercio online ( e-commerce ) de produtos de informática para casas e escritórios. Os clientes podem comprar mouses, monitores, teclados, computadores, laptops, cabos HDMI, fones de ouvido, cameras webcam, entre outros, através de um site online e recebem os produtos no conforto de suas casas.
Os produtos não são vendidos somente no Brasil, a Eletronic House está presente em diversos países da Europa e da América do Norte.
O Diretor de Produtos Global pediu ao Head de Design que desenvolvesse uma nova forma de finalizar a compra com cartão de crédito, sem a necessidade do cliente preencher manualmente todas as informações do cartão e que funcionasse em todos os países.
Depois de meses desenvolvendo esse dispositivo, o time de Desenvolvimento Backend entregou uma solução de pagamentos, na qual 90% das informações
  do formulário eram preenchido automaticamente.
O Head de Designer gostaria de medir a efetividade do novo dispositivo de preenchimento automático dos dados do cartão de crédito na página de vendas e reportar os resultados ao Diretor de Produtos Global, para concluir se a nova forma de pagamento é realmente melhor do que a antiga.
As duas páginas foram colocadas no ar e durante alguns meses e o time de Front-End desenvolveu uma automação que atribui um rótulo para cada cliente, mostrando qual a página de vendas aquele determinado cliente estava visualizando. Todos esses dados foram armazenados em um banco de dados e podem ser acessados pelos times da Electronic House.
### 2. O desafio
Depois de alguns meses de experimento, o time de Designers da Electronic House, precisa avaliar os resultados do experimento realizado, a fim de concluir qual era a forma de pagamento mais eficaz.
Porém, o time não possui as habilidades necessárias para avaliar os dados e concluir o experimento. Nesse contexto, você foi contratado como Cientista de Dados para ajudar o time de Designers a validar a efetividade do novo meio de pagamento, com mais confiança e rigidez na análise.
O entregável do seu trabalho é o seguinte:
### 1. Qual a melhor forma de pagamento: Preenchimento Manual ou Automático do formulário de dados do cartão de crédito?
### 3. Os dados
- Coluna Device:
I = Compra feita através do site
A= Compra feita através do App Coluna Group:
A = Preenchimento automático
B = Preenhimento manual

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [80]:
data = pd.read_csv('../teste_ab/dataset/ab_testing.csv')
data

,uid,country,gender,spent,purchases,date,group,device
0,11115722,MEX,F,1595.0,5.0,2016-03-08,GRP B,I
1,11122053,USA,M,498.0,2.0,2017-07-14,GRP B,I
2,11128688,USA,F,2394.0,6.0,2017-09-17,GRP A,I
3,11130578,USA,F,1197.0,3.0,2017-11-30,GRP A,I
4,11130759,ESP,M,1297.0,3.0,2018-01-10,GRP B,A
...,...,...,...,...,...,...,...,...
25225,59699156,CAN,M,3791.0,9.0,2017-05-25,GRP B,A
25226,59700351,MEX,M,698.0,2.0,2016-04-09,GRP A,I
25227,59702883,USA,M,1394.0,6.0,2016-07-24,GRP B,I
25228,59710537,MEX,F,2893.0,7.0,2016-10-29,GRP B,I


In [81]:
print('quantidade de linhas:{}'.format(data.shape[0]))
print('quantidade de colunas:{}'.format(data.shape[1]))

quantidade de linhas:25230
quantidade de colunas:8


In [82]:
data.dtypes

uid            int64
country       object
gender        object
spent        float64
purchases    float64
date          object
group         object
device        object
dtype: object

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25230 entries, 0 to 25229
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   uid        25230 non-null  int64  
 1   country    25229 non-null  object 
 2   gender     25229 non-null  object 
 3   spent      25229 non-null  float64
 4   purchases  25229 non-null  float64
 5   date       25229 non-null  object 
 6   group      25229 non-null  object 
 7   device     25229 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 1.5+ MB


In [84]:
data.describe()

,uid,spent,purchases
count,2.523000e+04,25229.000000,25229.000000
mean,3.551362e+07,1885.709660,4.543620
std,1.403681e+07,1360.705364,3.097174
min,5.970000e+02,99.000000,1.000000
25%,2.330667e+07,897.000000,2.000000
50%,3.553127e+07,1596.000000,4.000000
75%,4.772588e+07,2593.000000,6.000000
max,5.971054e+07,12170.000000,30.000000


In [85]:
data.isna().sum() 

uid          0
country      1
gender       1
spent        1
purchases    1
date         1
group        1
device       1
dtype: int64

In [86]:
data = data.dropna()

In [87]:
data.isna().sum() 

uid          0
country      0
gender       0
spent        0
purchases    0
date         0
group        0
device       0
dtype: int64

In [91]:
data.loc[:, 'purchases'] = data['purchases'].astype(int)
data.loc[:, 'device'] = data['device'].replace({'I': 0, 'A': 1})
data.loc[:, 'group'] = data['group'].replace({'GRP A': 0, 'GRP B': 1})
data.loc[:, 'date'] = pd.to_datetime(data['date'])


In [92]:
data.dtypes

uid                   int64
country              object
gender               object
spent               float64
purchases             int64
date         datetime64[ns]
group                 int64
device                int64
dtype: object

In [90]:
correlation_matrix = data.corr()


ValueError: could not convert string to float: 'MEX'